# Exercise 1.2
Application for counting the number of cars that go from city downtown to city centre in peak hours.

We are using an algorithm based on frame differencing and background subtraction techniques like in the first task, so the code will be reused.
The main difference is the counting of the total number of cars and cars per minute.

In [1]:
import cv2
import time

source_file_1 = './media/Traffic_Laramie_1.mp4'
source_file_2 = './media/Traffic_Laramie_2.mp4'

In [2]:
initial_frame = None
# Threshold to determine if a pixel is different from the initial frame
threshold = 10
# Minimum area of a contour to be considered
contour_area = 2500

# Number of cars detected
passing_cars = 0

# Threshold time to count a car in seconds
threshold_time = 1

# Last time a car was detected
last_time = time.time()

def analyze_video_source(
        videoSourceStr: str,
        setup_thresholds: bool = False,
    ):
    global passing_cars
    global initial_frame

    videoSource = cv2.VideoCapture(videoSourceStr)
    total_seconds = videoSource.get(cv2.CAP_PROP_FRAME_COUNT) / videoSource.get(cv2.CAP_PROP_FPS)

    print(f'Analyzing video source: {videoSourceStr}')
    print(f'Total seconds: {total_seconds}')
    while analyze_frame(videoSource, setup_thresholds):
        pass

    print(f'Total cars detected: {passing_cars}')
    print(f'Cars per minute: {round(passing_cars / total_seconds * 60)}')

    passing_cars = 0
    initial_frame = None
    videoSource.release()
    cv2.destroyAllWindows()

def analyze_frame(
        videoSource: cv2.VideoCapture,
        setup_thresholds: bool = False,
):
    """
    Analyze the video by frame
    Returns False if analysis is done, True if the analysis is still ongoing
    """
    ret, frame = videoSource.read()
    if not ret:
        if setup_thresholds:
            # Set the video on loop, this will allow us to continue setting the threshold
            videoSource.set(cv2.CAP_PROP_POS_FRAMES, 0)
            return True
        else:
            return False
    # As in the instructions the point of interest is roughly the last horizontal half of the video.
    # We will create a ROI (Region of Interest) to focus on that area.
    height, width, _ = frame.shape
    half_height = int(height/2)
    half_width = int(width/2)
    roi = frame[half_height:int(height - half_height / 1.7), 0:half_width]

    # Passing line coordinates
    passing_line = int(width/5)

    # Background subtraction
    fgmask = apply_background_subtraction(roi)

    # Frame difference
    global initial_frame
    if initial_frame is None:
        initial_frame = apply_gaussian_blur(roi)
        return True

    threshold_frame = apply_frame_difference(initial_frame, roi, threshold)

    # Combine the background subtraction and the threshold frame
    combined_mask = cv2.bitwise_and(fgmask, threshold_frame)

    # Get contours of the threshold frame
    contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw rectangles around the contours
    draw_contours(contours, frame, contour_area, height, passing_line)

    draw_over_frame(setup_thresholds, threshold, contour_area, frame, passing_line, half_height)

    return setup_controls(setup_thresholds)

def apply_background_subtraction(roi):
    """
    Apply background subtraction to the ROI and return the mask
    """
    fgmask = cv2.createBackgroundSubtractorMOG2().apply(roi)
    fgmask = cv2.erode(fgmask, None, iterations=2)
    fgmask = cv2.dilate(fgmask, None, iterations=2)
    return fgmask

def apply_frame_difference(initial_frame, frame, threshold):
    """
    Apply frame difference to the initial frame and the current frame
    Return the threshold frame
    """
    blur = apply_gaussian_blur(frame)
    delta_frame = cv2.absdiff(initial_frame, blur)
    threshold_frame = cv2.threshold(delta_frame, threshold, 255, cv2.THRESH_BINARY)[1]
    return threshold_frame

def apply_gaussian_blur(frame):
    """
    Apply Gaussian blur to the frame
    """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return cv2.GaussianBlur(gray, (25,25), 0)

def draw_contours(contours, frame, contour_area, height, passing_line):
    """
    Draw rectangles around the contours
    """
    half_height = int(height/2)
    for contour in contours:
        if cv2.contourArea(contour) < contour_area:
            continue
        x, y, w, h = cv2.boundingRect(contour)
        y = y + int(height/2)
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 1)
        contour_count_car(x, y, w, passing_line, frame, half_height)

def contour_count_car(x, y, w, passing_line, frame, half_height):
    """
    Check if the contour crosses the line and counts the car if the delta time is greater than the threshold
    """
    global last_time
    global passing_cars
    if passing_line > x and passing_line < x+w:
        if time.time() - last_time > threshold_time:
            last_time = time.time()
            cv2.line(frame, (passing_line, half_height + 20), (passing_line, half_height + 150), (0,255,0), 2)
            cv2.putText(frame, 'Car detected', (x,y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
            passing_cars += 1

def draw_over_frame(setup_thresholds, threshold, contour_area, frame, passing_line, half_height):
    """
    Draw text over the frame and show it
    """
    if setup_thresholds:
        cv2.putText(frame, f'Threshold: {threshold}', (10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
        cv2.putText(frame, f'Contour Area: {contour_area}', (10,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
        cv2.line(frame, (passing_line, half_height + 20), (passing_line, half_height + 150), (0,0,255), 2)
        cv2.putText(frame, f'Passing cars: {passing_cars}', (10,60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

        cv2.imshow('frame', frame)

def setup_controls(setup_thresholds):
    """
    Set up the control keys
    """
    key = cv2.waitKey(1)
    if key == ord('q'):
        return False

    if setup_thresholds:
        # If setting up trhesholds, w and s keys will increase and decrease the threshold value. a and d keys will do the same for the contour area
        if key == ord('w'):
            adjust_threshold(10)
        if key == ord('s'):
            adjust_threshold(-10)
        if key == ord('a'):
            adjust_contour_area(-100)
        if key == ord('d'):
            adjust_contour_area(100)
    return True

def adjust_threshold(value):
    """
    Adjust the threshold value
    """
    global threshold
    threshold += value

def adjust_contour_area(value):
    """
    Adjust the contour area value
    """
    global contour_area
    contour_area += value

In [3]:
analyze_video_source(source_file_1)
analyze_video_source(source_file_2)

Analyzing video source: ./media/Traffic_Laramie_1.mp4
Total seconds: 177.92
Total cars detected: 6
Cars per minute: 2.0233812949640293
Analyzing video source: ./media/Traffic_Laramie_2.mp4
Total seconds: 105.68
Total cars detected: 4
Cars per minute: 2.271006813020439
